In [2]:
import geopandas as gpd
bounds = gpd.read_file('kazan_no_water.geojson').to_crs(4326)
bounds.explore()

In [3]:
from iduedu import get_boundary, PublicTrasport, get_walk_graph, get_all_public_transport_graph, join_pt_walk_graph, \
    get_intermodal_graph

intermodal = get_intermodal_graph(polygon=bounds.union_all(),clip_by_bounds=True,transport_types=[PublicTrasport.BUS,PublicTrasport.TRAM,PublicTrasport.TROLLEYBUS])

# zone = gpd.GeoDataFrame(geometry=[bounds], crs=4326)




2025-05-13 18:37:49.652 | INFO     | iduedu.modules.drive_walk_builder:get_walk_graph:225 - Downloading walk graph from OSM, it may take a while for large territory ...


Parsing public transport routes:   0%|          | 0/155 [00:00<?, ?it/s]

Calculating the weights of the walk graph:   0%|          | 0/219740 [00:00<?, ?it/s]

2025-05-13 18:39:19.252 | INFO     | iduedu.modules.pt_walk_joiner:join_pt_walk_graph:51 - Composing intermodal graph...
2025-05-13 18:39:33.066 | WARNING  | iduedu.utils.utils:remove_weakly_connected_nodes:37 - Removing 3 nodes that form 1 trap components. These are groups where you can enter but can't exit (or vice versa). Keeping the largest strongly connected component (84717 nodes).


In [4]:
subways = gpd.read_file('subway_entrance_Kazan.geojson')

In [5]:
from objectnat import get_stepped_graph_coverage
from typing import Literal

import geopandas as gpd
import networkx as nx
import numpy as np
import pandas as pd
from pyproj.exceptions import CRSError
from shapely import Point, concave_hull

from objectnat.methods.utils.graph_utils import get_closest_nodes_from_gdf, remove_weakly_connected_nodes


weight = 90

gdf_to=subways.copy()
nx_graph=intermodal.copy()
weight_type="time_min"
weight_value_cutoff=weight
zone=bounds.copy()
step=5

if step is None:
    if weight_type == "length_meter":
        step = 100
    else:
        step = 1

original_crs = gdf_to.crs
try:
    local_crs = nx_graph.graph["crs"]
except KeyError as exc:
    raise ValueError("Graph does not have crs attribute") from exc

try:
    points = gdf_to.copy()
    points.to_crs(local_crs, inplace=True)
    zone.to_crs(local_crs, inplace=True)
except CRSError as e:
    raise CRSError(f"Graph crs ({local_crs}) has invalid format.") from e

old_len  = len(nx_graph)

if nx_graph.is_multigraph():
    if nx_graph.is_directed():
        nx_graph = nx.DiGraph(nx_graph)
    else:
        nx_graph = nx.Graph(nx_graph)
        
nx_graph = remove_weakly_connected_nodes(nx_graph)
new_len = len(nx_graph)

mapping = {old_label: new_label for new_label, old_label in enumerate(nx_graph.nodes())}
nx_graph = nx.relabel_nodes(nx_graph, mapping)

nodelist = list(nx_graph.nodes())

sparse_matrix = nx.to_scipy_sparse_array(nx_graph, weight=weight_type)
transposed_matrix = sparse_matrix.transpose()
reversed_graph = nx.from_scipy_sparse_array(
    transposed_matrix, edge_attribute=weight_type, create_using=type(nx_graph)
)
points.geometry = points.representative_point()

distances, nearest_nodes = get_closest_nodes_from_gdf(points, nx_graph)

points["nearest_node"] = nearest_nodes
points["distance"] = distances

dist = nx.multi_source_dijkstra_path_length(
    reversed_graph, nearest_nodes, weight=weight_type, cutoff=weight_value_cutoff
)

graph_points = pd.DataFrame(
    data=[{"node": node, "geometry": Point(data["x"], data["y"])} for node, data in nx_graph.nodes(data=True)]
)

nearest_nodes = pd.DataFrame.from_dict(dist, orient="index", columns=["dist"]).reset_index()

graph_nodes_gdf = gpd.GeoDataFrame(
    graph_points.merge(nearest_nodes, left_on='node', right_on='index', how="left").reset_index(drop=True),
    geometry="geometry",
    crs=local_crs,
)
graph_nodes_gdf["dist"] = np.minimum(np.ceil(graph_nodes_gdf["dist"] / step) * step, weight_value_cutoff)
voronois = gpd.GeoDataFrame(geometry=graph_nodes_gdf.voronoi_polygons(), crs=local_crs)

zone_coverages = voronois.sjoin(graph_nodes_gdf).dissolve(by="dist", as_index=False,dropna=False)
zone_coverages:gpd.GeoDataFrame
zone_coverages  = zone_coverages.clip(zone,keep_geom_type=True)

In [7]:
zone_coverages.to_file('stepped_zone_coverage_Kazan.geojson')

In [10]:
m1 = zone_coverages.explore(column="dist",cmap="viridis",tiles='cartoDB Positron')
subways.explore(m=m1,color='red')
m1.save('stepped_zone_coverage_Kazan.html')

In [9]:
subways.explore()

In [ ]:
import networkx as nx

for node,data in nx_graph.nodes(data=True):
    nx_graph.nodes(data=True)[node]['x'] = float(data['x'])
    nx_graph.nodes(data=True)[node]['y'] = float(data['y'])

nx.write_gml(nx_graph,'intermodal.gml',stringizer=lambda x:str(x))


In [ ]:
for node,data in reversed_graph.nodes(data=True):
    reversed_graph.nodes(data=True)[node]['x'] = float(nx_graph.nodes(data=True)[node]['x'])
    reversed_graph.nodes(data=True)[node]['y'] = float(nx_graph.nodes(data=True)[node]['y'])

nx.write_gml(reversed_graph,'intermodal_reversed.gml',stringizer=lambda x:str(x))